In [ ]:
!pip install -r requirements.txt

  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.7.0
    Uninstalling accelerate-1.7.0:
      Successfully uninstalled accelerate-1.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
!pip install pyngrok

In [ ]:
# In your Colab notebook, click the key icon (🔑) on the left-hand side toolbar.
# Click "Add a new secret".
# For the Name, enter NGROK_AUTHTOKEN.

In [ ]:
from google.colab import userdata
userdata.get('NGROK_AUTH_TOKEN')

NGROK_AUTH_TOKEN="sdfgyhuioplkjhgfd"

In [ ]:
# !pip install flask flask-cors torch transformers pillow requests numpy

In [ ]:
# --- Core Setup (Move this BEFORE ngrok functions) ---
import logging
import os

# Set up logging first
LOGGING_LEVEL = os.getenv("LOGGING_LEVEL", "INFO").upper()
logging.basicConfig(level=getattr(logging, LOGGING_LEVEL, logging.INFO),
                   format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("flask_qwen_vl_app")

# --- Ngrok Setup ---
from pyngrok import ngrok
import subprocess
import sys

NGROK_AUTH_TOKEN = os.getenv("NGROK_AUTH_TOKEN")
NGROK_STATIC_DOMAIN = "firmly-chitah-haha.ngrok-free.app"
FLASK_PORT = int(os.getenv("FLASK_PORT", 7860))  # Define FLASK_PORT before using it

def install_ngrok():
    """Install pyngrok if not already installed"""
    try:
        import pyngrok
        logger.info("pyngrok is already installed")
    except ImportError:
        logger.info("Installing pyngrok...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "pyngrok"])
        import pyngrok
        logger.info("pyngrok installed successfully")

def setup_ngrok():
    """Setup ngrok tunnel with static domain"""
    try:
        # Install ngrok if needed
        install_ngrok()

        # Set auth token if provided
        if NGROK_AUTH_TOKEN:
            ngrok.set_auth_token(NGROK_AUTH_TOKEN)
            logger.info("ngrok auth token set")
        else:
            logger.warning("No NGROK_AUTH_TOKEN provided - ngrok may not work without authentication")

        # Kill any existing tunnels
        ngrok.kill()

        # Create new tunnel with static domain
        if NGROK_STATIC_DOMAIN:
            # Use static domain
            public_tunnel = ngrok.connect(FLASK_PORT, hostname=NGROK_STATIC_DOMAIN)
            public_url = f"https://{NGROK_STATIC_DOMAIN}"
            logger.info(f"🌐 Using static domain: {NGROK_STATIC_DOMAIN}")
        else:
            # Fallback to random domain
            public_tunnel = ngrok.connect(FLASK_PORT)
            public_url = public_tunnel.public_url
            logger.info(f"🌐 Using random domain")

        logger.info(f"🔗 Your Flask app is now accessible at: {public_url}")

        return public_url

    except Exception as e:
        logger.error(f"Failed to setup ngrok: {e}")
        logger.info("App will run locally only")
        return None

In [ ]:
# --- Flask API Imports ---
from flask import Flask, request, jsonify, Response, stream_with_context
from flask_cors import CORS

# --- Original Core Logic Imports (Now used by Flask) ---
import logging
import re
from PIL import Image
import requests
from io import BytesIO
import base64
import os
import uuid
import json
import time
from threading import Thread, Lock, Semaphore
from collections import deque
import csv
import datetime
import torch
from transformers import AutoProcessor, TextIteratorStreamer, Qwen2_5_VLForConditionalGeneration
import numpy # For qwen_model_handler

# --- Global Configuration ---
LOGGING_LEVEL = os.getenv("LOGGING_LEVEL", "INFO").upper()
FLASK_PORT = int(os.getenv("FLASK_PORT", 7860))
FLASK_DEBUG = os.getenv("FLASK_DEBUG", "True").lower() == 'true'
PRELOAD_DEFAULT_MODEL = os.getenv("PRELOAD_DEFAULT_MODEL", "false").lower() == "true"
MAX_CONCURRENT_REQUESTS = int(os.getenv("MAX_CONCURRENT_REQUESTS", 3))

# --- Model Configurations ---
MODEL_CONFIGS = [
    { "id": "unsloth/Qwen2.5-VL-3B-Instruct-unsloth-bnb-4bit", "display_name": "Qwen2.5-VL 3B (Instruct, 4-bit Unsloth)" },
    { "id": "unsloth/Qwen2.5-VL-7B-Instruct-unsloth-bnb-4bit", "display_name": "Qwen2.5-VL 7B (Instruct, 4-bit Unsloth)" },
    { "id": "unsloth/Qwen2.5-VL-32B-Instruct-unsloth-bnb-4bit", "display_name": "Qwen2.5-VL 32B (Instruct, 4-bit Unsloth)" }
]
DEFAULT_MODEL_CONFIG = MODEL_CONFIGS[1]
DEFAULT_MODEL_ID = DEFAULT_MODEL_CONFIG["id"]
DEFAULT_MODEL_DISPLAY_NAME = DEFAULT_MODEL_CONFIG["display_name"]
MODEL_DISPLAY_NAME_TO_ID_MAP = {config["display_name"]: config["id"] for config in MODEL_CONFIGS}

DEFAULT_TEMPERATURE = float(os.getenv("DEFAULT_TEMPERATURE", 0.1))

# --- Performance & Loading Defaults ---
DEFAULT_LOAD_PARAMS = {
    "load_in_4bit": True,
    "bnb_4bit_quant_type": os.getenv("DEFAULT_BNB_4BIT_QUANT_TYPE", "nf4"),
    "bnb_4bit_use_double_quant": os.getenv("DEFAULT_BNB_4BIT_USE_DOUBLE_QUANT", "true").lower() == 'true',
    "bnb_4bit_compute_dtype_str": os.getenv("DEFAULT_BNB_4BIT_COMPUTE_DTYPE", "auto"),
    "llm_int8_enable_fp32_cpu_offload": os.getenv("DEFAULT_LLM_INT8_ENABLE_FP32_CPU_OFFLOAD", "false").lower() == 'true',
}

# --- Core Setup ---
logging.basicConfig(level=getattr(logging, LOGGING_LEVEL, logging.INFO), format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("flask_qwen_vl_app")
LOGS_DIRECTORY = os.path.join(os.getcwd(), 'logs')
TEMP_IMAGE_DIRECTORY = os.path.join(LOGS_DIRECTORY, 'temp_images')
os.makedirs(TEMP_IMAGE_DIRECTORY, exist_ok=True)
USAGE_LOG_FILE = os.path.join(LOGS_DIRECTORY, 'usage_log_flask.csv')
USAGE_LOG_HEADERS = ["timestamp", "user_identifier", "request_uuid", "model_id", "temperature", "system_prompt_key_used", "prompt_text", "image_provided", "image_details", "ai_response_length", "ai_response_preview", "ttft_ms", "generation_time_ms", "total_stream_handler_time_ms", "error_message", "load_params_used"]
usage_log_lock = Lock()
loaded_models_cache = {}

# --- Concurrency & Queue Management ---
logger.info(f"Setting max concurrent requests to: {MAX_CONCURRENT_REQUESTS}")
request_semaphore = Semaphore(MAX_CONCURRENT_REQUESTS)
queue_lock = Lock()
request_queue = deque()  # Stores request_uuids of waiting requests
active_requests = {}     # Maps request_uuid to its start time for active requests


def init_usage_log_csv():
    with usage_log_lock:
        if not os.path.exists(USAGE_LOG_FILE):
            with open(USAGE_LOG_FILE, mode='w', newline='', encoding='utf-8') as f:
                csv.writer(f).writerow(USAGE_LOG_HEADERS)

def log_usage_data(**kwargs):
    with usage_log_lock:
        row = [kwargs.get(h, "") for h in USAGE_LOG_HEADERS]
        with open(USAGE_LOG_FILE, mode='a', newline='', encoding='utf-8') as f:
            csv.writer(f).writerow(row)

def _load_image_from_url(url):
    try:
        if url.startswith('http'):
            response = requests.get(url, stream=True, timeout=15)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content))
        elif os.path.exists(url):
            img = Image.open(url)
        else: return None
        return img.convert('RGB') if img.mode != 'RGB' else img
    except Exception as e:
        logger.error(f"Error loading image {url[:60]}...: {e}")
        return None

def extract_pil_images_from_messages(messages):
    pil_images = []
    if not messages: return pil_images
    for msg in messages:
        if msg.get("role") == "user" and isinstance(msg.get("content"), list):
            for item in msg["content"]:
                if item.get("type") == "image_url":
                    url_data = item.get("image_url", {})
                    url = url_data.get("url")
                    if url and (img := _load_image_from_url(url)):
                        pil_images.append(img)
    return pil_images

DEFAULT_SYSTEM_PROMPTS = {
    "ocr_digit_only": "You are an expert Optical Character Recognition (OCR) assistant. Your sole task is to meticulously extract ONLY THE DIGITS (0-9) visible in the provided image. Present these digits clearly, for example, separated by spaces or newlines. If no digits are found, explicitly state 'No digits found'. Do not provide any other text, explanation, or commentary. If no image is provided, state 'Please provide an image for digit extraction.'",
    "ocr_general": "You are an expert OCR assistant. Your primary task is to meticulously extract ALL text, numbers, and symbols visible in any provided image or described scene. Transcribe the text exactly as it appears. Only output the extracted text. If no image is clearly referenced or uploaded, state that you need an image or image URL to perform OCR.",
    "ocr_receipt": "You are an expert OCR assistant specializing in receipts. Extract all items, quantities, and prices. Also identify the store name, date, and total amount. Present the information in a structured format if possible.",
    "chat_general_helper": "You are a helpful AI assistant. Analyze the provided image and respond to the user's query."
}
PRIMARY_DEFAULT_SYSTEM_PROMPT_KEY = "ocr_digit_only"

def get_dtype_from_string(dtype_str: str):
    if dtype_str == "bfloat16": return torch.bfloat16
    if dtype_str == "float16": return torch.float16
    if dtype_str == "float32": return torch.float32
    return "auto"

def get_model_and_processor(model_id: str, load_in_4bit: bool, bnb_4bit_quant_type: str, bnb_4bit_use_double_quant: bool, bnb_4bit_compute_dtype_str: str, llm_int8_enable_fp32_cpu_offload: bool):
    param_tuple = (model_id, f"4bit-{load_in_4bit}", f"quant-{bnb_4bit_quant_type}", f"doubleq-{bnb_4bit_use_double_quant}", f"compute-{bnb_4bit_compute_dtype_str}", f"offload-{llm_int8_enable_fp32_cpu_offload}")
    cache_key = "_".join(param_tuple)
    if cache_key in loaded_models_cache: return loaded_models_cache[cache_key]

    logger.info(f"Initiating load for model '{model_id}'. Cache key: {cache_key}")
    model_kwargs = {"trust_remote_code": True}
    if torch.cuda.is_available():
        model_kwargs["device_map"] = "auto"
        actual_compute_dtype = get_dtype_from_string(bnb_4bit_compute_dtype_str)
        if actual_compute_dtype == "auto":
            actual_compute_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16

        model_kwargs.update({
            "load_in_4bit": load_in_4bit,
            "bnb_4bit_quant_type": bnb_4bit_quant_type,
            "bnb_4bit_use_double_quant": bnb_4bit_use_double_quant,
            "bnb_4bit_compute_dtype": actual_compute_dtype
        })
        if llm_int8_enable_fp32_cpu_offload:
            model_kwargs["llm_int8_enable_fp32_cpu_offload"] = True
    else:
        logger.warning(f"CUDA NOT available. Model '{model_id}' will be loaded on CPU.")
        model_kwargs.update({"device_map": "cpu", "torch_dtype": torch.float32})

    try:
        model = Qwen2_5_VLForConditionalGeneration.from_pretrained(model_id, **model_kwargs)
        processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
        loaded_models_cache[cache_key] = (model, processor)
        logger.info(f"Successfully loaded model and processor for: {model_id}")
        return model, processor
    except Exception as e:
        logger.error(f"Error loading model '{model_id}': {e}", exc_info=True)
        if cache_key in loaded_models_cache: del loaded_models_cache[cache_key]
        raise e

def generate_chat_response_blocking(model_id_param: str, messages_for_model, temperature, **load_params):
    model, processor = get_model_and_processor(model_id_param, **load_params)
    pil_images = extract_pil_images_from_messages(messages_for_model)
    text_prompt = processor.apply_chat_template(messages_for_model, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text_prompt], images=pil_images or None, return_tensors="pt").to(model.device)

    generation_kwargs = dict(inputs, max_new_tokens=2048, temperature=max(temperature, 0.01))
    outputs = model.generate(**generation_kwargs)

    # Decode only the newly generated tokens, excluding the prompt
    generated_tokens = outputs[:, inputs.input_ids.shape[-1]:]
    response_text = processor.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    return response_text

def generate_chat_response_stream(model_id_param: str, messages_for_model, temperature, **load_params):
    model, processor = get_model_and_processor(model_id_param, **load_params)
    pil_images = extract_pil_images_from_messages(messages_for_model)
    text_prompt = processor.apply_chat_template(messages_for_model, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text_prompt], images=pil_images or None, return_tensors="pt").to(model.device)
    streamer = TextIteratorStreamer(processor.tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=2048, temperature=max(temperature, 0.01))
    Thread(target=model.generate, kwargs=generation_kwargs).start()

    buffer = ""
    for chunk in streamer:
        if chunk: buffer += chunk
        if ' ' in buffer or '\n' in buffer or len(buffer) > 5:
            yield buffer
            buffer = ""
    if buffer: yield buffer

def prepare_qwen_messages_for_model(current_message_dict, system_prompt_text, request_uuid, system_prompt_key):
    qwen_messages = [{"role": "system", "content": system_prompt_text}]
    logged_image_path = None

    current_user_prompt_text = current_message_dict.get("text", "")
    current_qwen_user_content_parts = []
    image_part_added = False

    if current_message_dict.get("files"):
        temp_image_path = current_message_dict["files"][0]
        current_qwen_user_content_parts.append({"type": "image_url", "image_url": {"url": temp_image_path}})
        image_part_added = True
        if pil_image_to_log := _load_image_from_url(temp_image_path):
            logged_image_path = os.path.join(LOGS_DIRECTORY, f"log_img_{request_uuid}.png")
            pil_image_to_log.save(logged_image_path)

    if current_user_prompt_text.strip():
        current_qwen_user_content_parts.append({"type": "text", "text": current_user_prompt_text.strip()})
    elif image_part_added:
        default_text = "ocr digit" if system_prompt_key == "ocr_digit_only" else "Describe the image."
        current_qwen_user_content_parts.append({"type": "text", "text": default_text})

    if current_qwen_user_content_parts:
        qwen_messages.append({"role": "user", "content": current_qwen_user_content_parts})

    return qwen_messages, logged_image_path

def process_chat_request_blocking(request_uuid, **args):
    actual_model_id = MODEL_DISPLAY_NAME_TO_ID_MAP.get(args['model_display_name'], DEFAULT_MODEL_ID)
    system_prompt_text = DEFAULT_SYSTEM_PROMPTS.get(args['system_prompt_key'], DEFAULT_SYSTEM_PROMPTS[PRIMARY_DEFAULT_SYSTEM_PROMPT_KEY])

    if not args['current_message'].get("text", "").strip() and not args['current_message'].get("files", []):
        return {"error": "Please provide some input or an image."}, 400

    messages_for_model, logged_image_path = prepare_qwen_messages_for_model(args['current_message'], system_prompt_text, request_uuid, args['system_prompt_key'])
    if len(messages_for_model) < 2:
        return {"error": "Could not form a valid message to send."}, 400

    load_params = {k: v for k, v in args.items() if k in DEFAULT_LOAD_PARAMS}
    user_prompt_text_for_log = next((item["text"] for item in messages_for_model[-1]["content"] if item["type"] == "text"), "")
    log_payload = {"timestamp": datetime.datetime.now(datetime.timezone.utc).isoformat(), "user_identifier": "api_user", "request_uuid": request_uuid, "model_id": actual_model_id, "temperature": args['temperature'], "system_prompt_key_used": args['system_prompt_key'], "prompt_text": user_prompt_text_for_log, "image_provided": bool(logged_image_path), "image_details": logged_image_path or "none", "load_params_used": json.dumps(load_params)}

    full_ai_response = ""
    start_time = time.monotonic()
    try:
        full_ai_response = generate_chat_response_blocking(
            model_id_param=actual_model_id,
            messages_for_model=messages_for_model,
            temperature=args['temperature'],
            **load_params
        )
        end_time = time.monotonic()
        total_time_ms = round((end_time - start_time) * 1000)

        log_payload.update({"ai_response_length": len(full_ai_response), "ai_response_preview": full_ai_response[:200], "ttft_ms": total_time_ms, "generation_time_ms": total_time_ms, "total_stream_handler_time_ms": total_time_ms, "error_message": ""})

        response_data = {"content": full_ai_response, "request_uuid": request_uuid, "model_id": actual_model_id}
        return response_data, 200

    except Exception as e:
        logger.error(f"Error during blocking model generation for {request_uuid}: {e}", exc_info=True)
        total_time_ms = round((time.monotonic() - start_time) * 1000)
        log_payload.update({"error_message": str(e)[:200], "total_stream_handler_time_ms": total_time_ms})
        return {"error": f"Error during model generation: {str(e)[:200]}", "request_uuid": request_uuid}, 500
    finally:
        log_usage_data(**log_payload)


def process_chat_request_stream(request_uuid, **args):
    actual_model_id = MODEL_DISPLAY_NAME_TO_ID_MAP.get(args['model_display_name'], DEFAULT_MODEL_ID)
    system_prompt_text = DEFAULT_SYSTEM_PROMPTS.get(args['system_prompt_key'], DEFAULT_SYSTEM_PROMPTS[PRIMARY_DEFAULT_SYSTEM_PROMPT_KEY])

    if not args['current_message'].get("text", "").strip() and not args['current_message'].get("files", []):
        yield f"data: {json.dumps({'error': 'Please provide some input or an image.'})}\n\n"
        return

    messages_for_model, logged_image_path = prepare_qwen_messages_for_model(args['current_message'], system_prompt_text, request_uuid, args['system_prompt_key'])
    if len(messages_for_model) < 2:
        yield f"data: {json.dumps({'error': 'Could not form a valid message to send.'})}\n\n"
        return

    load_params = {k: v for k, v in args.items() if k in DEFAULT_LOAD_PARAMS}
    user_prompt_text_for_log = next((item["text"] for item in messages_for_model[-1]["content"] if item["type"] == "text"), "")
    log_payload = {"timestamp": datetime.datetime.now(datetime.timezone.utc).isoformat(), "user_identifier": "api_user", "request_uuid": request_uuid, "model_id": actual_model_id, "temperature": args['temperature'], "system_prompt_key_used": args['system_prompt_key'], "prompt_text": user_prompt_text_for_log, "image_provided": bool(logged_image_path), "image_details": logged_image_path or "none", "load_params_used": json.dumps(load_params)}

    full_ai_response = ""
    stream_start_time = time.monotonic()
    first_chunk_time = None
    try:
        stream_gen = generate_chat_response_stream(
            model_id_param=actual_model_id,
            messages_for_model=messages_for_model,
            temperature=args['temperature'],
            **load_params
        )
        for chunk in stream_gen:
            if first_chunk_time is None: first_chunk_time = time.monotonic()
            if chunk:
                full_ai_response += chunk
                yield f"data: {json.dumps({'content': full_ai_response})}\n\n" # Yield cumulative response

        total_time_ms = round((time.monotonic() - stream_start_time) * 1000)
        ttft_ms = round((first_chunk_time - stream_start_time) * 1000) if first_chunk_time else total_time_ms
        gen_time_ms = round((time.monotonic() - first_chunk_time) * 1000) if first_chunk_time else 0
        log_payload.update({"ai_response_length": len(full_ai_response), "ai_response_preview": full_ai_response[:200], "ttft_ms": ttft_ms, "generation_time_ms": gen_time_ms, "total_stream_handler_time_ms": total_time_ms, "error_message": ""})
    except Exception as e:
        logger.error(f"Error during model generation for {request_uuid}: {e}", exc_info=True)
        total_time_ms = round((time.monotonic() - stream_start_time) * 1000)
        yield f"data: {json.dumps({'error': f'ERROR: {str(e)[:200]}'})}\n\n"
        log_payload.update({"error_message": str(e)[:200], "total_stream_handler_time_ms": total_time_ms})
    finally:
        log_usage_data(**log_payload)
        # Signal end of stream
        yield f"data: {json.dumps({'event': 'done'})}\n\n"

def preload_models_on_startup():
    if PRELOAD_DEFAULT_MODEL:
        logger.info(f"Attempting to preload default model: {DEFAULT_MODEL_ID}...")
        try:
            get_model_and_processor(DEFAULT_MODEL_ID, **DEFAULT_LOAD_PARAMS)
        except Exception as e:
            logger.error(f"Failed to preload default model {DEFAULT_MODEL_ID}: {e}", exc_info=True)

app = Flask(__name__)
CORS(app)

@app.route('/', methods=['GET'])
def health_check(): return jsonify({"status": "ok", "message": "Qwen-VL API is running."})

@app.route('/api/queue_status', methods=['GET'])
def queue_status():
    request_uuid = request.args.get('request_uuid')
    with queue_lock:
        # Create copies to avoid issues with modifying the collections while iterating
        active_list = list(active_requests.keys())
        waiting_list = list(request_queue)

    status = {
        "max_concurrent_requests": MAX_CONCURRENT_REQUESTS,
        "active_requests_count": len(active_list),
        "queued_requests_count": len(waiting_list),
        "active_requests": active_list,
        "queued_requests": waiting_list,
    }

    if request_uuid:
        if request_uuid in active_list:
            status['your_status'] = 'processing'
            status['your_queue_position'] = 0
        elif request_uuid in waiting_list:
            try:
                # +1 for 1-based indexing
                position = waiting_list.index(request_uuid) + 1
                status['your_status'] = 'queued'
                status['your_queue_position'] = position
            except ValueError:
                # Should not happen due to the lock, but as a safeguard
                status['your_status'] = 'unknown'
                status['your_queue_position'] = -1
        else:
            status['your_status'] = 'completed_or_unknown'
            status['your_queue_position'] = -1

    return jsonify(status)

@app.route('/api/chat', methods=['POST'])
def chat_endpoint():
    temp_image_path = None
    request_uuid = str(uuid.uuid4())

    try:
        stream = request.form.get('stream', 'true').lower() == 'true'

        image_file = request.files.get('image_file')
        temp_files_list = []
        if image_file and image_file.filename:
            temp_image_path = os.path.join(TEMP_IMAGE_DIRECTORY, f"api_img_{uuid.uuid4()}_{image_file.filename}")
            image_file.save(temp_image_path)
            temp_files_list.append(temp_image_path)

        def get_form_val(key, default, converter):
            val = request.form.get(key)
            return converter(val) if val is not None else default

        load_params = {
            "load_in_4bit": get_form_val("load_in_4bit", DEFAULT_LOAD_PARAMS["load_in_4bit"], lambda v: v.lower() == 'true'),
            "bnb_4bit_quant_type": request.form.get("bnb_4bit_quant_type", DEFAULT_LOAD_PARAMS["bnb_4bit_quant_type"]),
            "bnb_4bit_use_double_quant": get_form_val("bnb_4bit_use_double_quant", DEFAULT_LOAD_PARAMS["bnb_4bit_use_double_quant"], lambda v: v.lower() == 'true'),
            "bnb_4bit_compute_dtype_str": request.form.get("bnb_4bit_compute_dtype_str", DEFAULT_LOAD_PARAMS["bnb_4bit_compute_dtype_str"]),
            "llm_int8_enable_fp32_cpu_offload": get_form_val("llm_int8_enable_fp32_cpu_offload", DEFAULT_LOAD_PARAMS["llm_int8_enable_fp32_cpu_offload"], lambda v: v.lower() == 'true')
        }

        args = {
            "current_message": {"text": request.form.get("message", ""), "files": temp_files_list},
            "model_display_name": request.form.get("model_display_name", DEFAULT_MODEL_DISPLAY_NAME),
            "system_prompt_key": request.form.get("system_prompt_key", PRIMARY_DEFAULT_SYSTEM_PROMPT_KEY),
            "temperature": get_form_val("temperature", DEFAULT_TEMPERATURE, float),
            **load_params
        }

        if stream:
            def generate_stream_response():
                with queue_lock:
                    request_queue.append(request_uuid)
                    position = len(request_queue)
                    active_count = len(active_requests)

                yield f"data: {json.dumps({'event': 'queued', 'request_uuid': request_uuid, 'queue_position': position, 'active_requests': active_count})}\n\n"

                request_semaphore.acquire()
                try:
                    with queue_lock:
                        if request_uuid in request_queue:
                            request_queue.remove(request_uuid)
                        active_requests[request_uuid] = time.monotonic()

                    yield f"data: {json.dumps({'event': 'processing_started', 'request_uuid': request_uuid})}\n\n"
                    yield from process_chat_request_stream(request_uuid, **args)
                finally:
                    with queue_lock:
                        active_requests.pop(request_uuid, None)
                    request_semaphore.release()

            return Response(stream_with_context(generate_stream_response()), mimetype='text/event-stream')
        else: # Blocking request
            with queue_lock:
                request_queue.append(request_uuid)

            request_semaphore.acquire()
            try:
                with queue_lock:
                    if request_uuid in request_queue:
                        request_queue.remove(request_uuid)
                    active_requests[request_uuid] = time.monotonic()

                response_data, status_code = process_chat_request_blocking(request_uuid, **args)
                return jsonify(response_data), status_code
            finally:
                with queue_lock:
                    active_requests.pop(request_uuid, None)
                request_semaphore.release()

    except Exception as e:
        logger.error(f"Unhandled error in chat endpoint for {request_uuid}: {e}", exc_info=True)
        with queue_lock:
            if request_uuid in request_queue:
                request_queue.remove(request_uuid)
        return jsonify({"error": "An unexpected server error occurred.", "request_uuid": request_uuid}), 500
    finally:
        if temp_image_path and os.path.exists(temp_image_path):
            os.remove(temp_image_path)

if __name__ == '__main__':
    init_usage_log_csv()

    # Setup ngrok (optional)
    public_url = setup_ngrok()

    # Preload models if requested
    preload_models_on_startup()

    logger.info("Starting Qwen-VL Flask API Server...")
    logger.info(f"Server will run on http://0.0.0.0:{FLASK_PORT}")
    if public_url:
        logger.info(f"Public URL: {public_url}")

    app.run(host="0.0.0.0", port=FLASK_PORT, debug=FLASK_DEBUG, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:7860
 * Running on http://172.28.0.12:7860
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Jun/2025 19:43:05] "GET / HTTP/1.1" 200 -
